In [4]:
from pbpstats.client import Client
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network
import time
from nba_api.stats.endpoints import leaguelineupviz, commonteamroster

settings = {
    "dir": "WNBA/response_data",
    "Games": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "EnhancedPbp": {"source": "file", "data_provider": "data_nba"}
}

client = Client(settings)
s = client.Season("wnba", "2022", "Regular Season")

# Read dictionary keyed by team names and valued by team IDs
with open('WNBA_teams_dict.txt','r') as f:
    team_names_dict = eval(f.read())

team_ids_dict = {v:k for k,v in team_names_dict.items()}

In [22]:
# Save play-by-play data in directory (when source is web)
settings = {
    "dir": "WNBA/response_data",
    "Games": {"source": "web", "data_provider": "data_nba"},
    "Possessions": {"source": "web", "data_provider": "data_nba"},
    "EnhancedPbp": {"source": "web", "data_provider": "data_nba"}
}

client = Client(settings)
s = client.Season("wnba", "2022", "Regular Season")

for game in s.games.items:
    client.Game(game.game_id)

In [53]:
# Create dictionary keyed by team names and valued by team IDs
import json
from nba_api.stats.endpoints import leaguestandingsv3

l = leaguestandingsv3.LeagueStandingsV3("10","2022")
standings = l.get_data_frames()[0]
standings = standings[['TeamID','TeamName']]
team_names_and_ids = []
for row in standings.iterrows():
    team_names_and_ids.append(list(row[1]))

team_names_dict = {prop[1]:prop[0] for prop in team_names_and_ids}
with open('WNBA_teams_dict.txt','w') as file:
    file.write(json.dumps(team_names_dict))


In [14]:
# Get 2-player lineup data to normalize edge weights

from nba_api.stats.endpoints import leaguelineupviz, commonteamroster

l = leaguelineupviz.LeagueLineupViz(group_quantity=2, season='2022', rank='N', \
        measure_type_detailed_defense='Base', minutes_min=0, per_mode_detailed='PerGame',league_id_nullable='10')

line = l.get_data_frames()[0]
line

,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,MIN,OFF_RATING,DEF_RATING,NET_RATING,PACE,TS_PCT,...,PCT_PTS_2PT_MR,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_PAINT,PCT_AST_FGM,PCT_UAST_FGM,OPP_FG3_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT
0,-1628276-1628932-,K. Plum - A. Wilson,1611661319,LVA,1029.0,112.1,98.3,13.8,100.01,0.590,...,0.121,0.137,0.179,0.397,0.649,0.351,0.344,0.494,0.215,0.165
1,-1628276-1629498-,K. Plum - J. Young,1611661319,LVA,988.7,112.6,101.8,10.8,100.23,0.588,...,0.120,0.139,0.178,0.388,0.633,0.367,0.370,0.518,0.219,0.161
2,-1628932-1629498-,A. Wilson - J. Young,1611661319,LVA,933.0,112.4,100.1,12.3,99.73,0.590,...,0.130,0.131,0.175,0.397,0.644,0.356,0.356,0.508,0.214,0.161
3,-203827-1629477-,N. Howard - S. Ionescu,1611661313,NYL,930.4,102.9,100.9,1.9,95.60,0.569,...,0.082,0.098,0.136,0.425,0.725,0.275,0.342,0.473,0.277,0.151
4,-204319-1627668-,J. Loyd - B. Stewart,1611661328,SEA,908.2,107.8,95.1,12.7,98.07,0.571,...,0.100,0.163,0.151,0.412,0.766,0.234,0.318,0.487,0.199,0.199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,-203825-1631072-,K. McBride - E. Cunane,1611661324,MIN,0.0,0.0,0.0,0.0,0.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1102,-201913-1630097-,K. Vaughn - K. Charles,1611661330,ATL,0.0,0.0,0.0,0.0,0.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1103,-203400-1628317-,S. Diggins-Smith - K. Samuelson,1611661317,PHO,0.0,0.0,0.0,0.0,0.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1104,-201480-1630452-,S. Fowles - R. Davis,1611661324,MIN,0.0,0.0,0.0,0.0,0.00,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [11]:
# Create weighted scoring networks

networks = {}
IDs = {}
names = {}
for team in list(team_names_dict.values()):
    
    r = commonteamroster.CommonTeamRoster(season="2022", team_id=team,league_id_nullable='10')
    roster = r.get_data_frames()[0]
    roster = roster[['PLAYER_ID','PLAYER']]

    IDs[team] = roster['PLAYER_ID'].tolist()
    names[team] = roster['PLAYER'].tolist()
    networks[team] = nx.complete_graph(IDs[team])
    nx.set_edge_attributes(networks[team],0,'weight')

for season_game in s.games.items:
    game = client.Game(season_game.game_id)
    team1 = season_game.data['home_team_id']
    team2 = season_game.data['away_team_id']

    for shot in game.enhanced_pbp.fgms:
        shooting_team = shot.data['team_id']
        if shot.data["player1_id"] in IDs[shooting_team]:
            shooter = shot.data["player1_id"]
            for player in shot.current_players[shooting_team]:
                if player != shooter and player in IDs[shooting_team]:
                    networks[shooting_team][player][shooter]['weight'] += shot.shot_data["ShotValue"]
    
    # Increase the appropriate edges for each made free throw
    for ft in game.enhanced_pbp.ftas:
        shooting_team = ft.data['team_id']
        if ft.is_made and ft.data['player1_id'] in IDs[shooting_team]:
            shooter = ft.data["player1_id"]
            for player in ft.event_for_efficiency_stats.current_players[shooting_team]:
                if player != shooter and player in IDs[shooting_team]:
                    networks[shooting_team][player][shooter]['weight'] += 1
    print(team_ids_dict[team1],' versus ',team_ids_dict[team2])

# Divide edge weights by minutes played
for G in networks.values():
    for u,v,d in G.edges(data=True):
        if not line.loc[line['GROUP_ID'].str.contains(str(v)) & \
                line['GROUP_ID'].str.contains(str(u)),'MIN'].empty:
            w = float(G[u][v]['weight'])
            try:
                G[u][v]['weight'] = w / float(line.loc[line['GROUP_ID'].str.contains(str(u)) & \
                    line['GROUP_ID'].str.contains(str(v)),'MIN'].iat[0])
            except:
                pass

Mystics  versus  Fever
Sky  versus  Sparks
Mercury  versus  Aces
Storm  versus  Lynx
Liberty  versus  Sun
Wings  versus  Dream
Fever  versus  Sparks
Lynx  versus  Mystics
Aces  versus  Storm
Mystics  versus  Aces
Fever  versus  Lynx
Dream  versus  Sparks
Sky  versus  Liberty
Mercury  versus  Storm
Mystics  versus  Wings
Dream  versus  Aces
Liberty  versus  Fever
Storm  versus  Mercury
Sun  versus  Sparks
Lynx  versus  Sky
Liberty  versus  Wings
Fever  versus  Dream
Liberty  versus  Sun
Fever  versus  Dream
Wings  versus  Mystics
Aces  versus  Mercury
Sparks  versus  Lynx
Storm  versus  Sky
Aces  versus  Lynx
Mercury  versus  Wings
Sun  versus  Fever
Dream  versus  Mystics
Storm  versus  Sparks
Aces  versus  Mercury
Wings  versus  Lynx
Fever  versus  Sun
Mystics  versus  Sky
Aces  versus  Sparks
Sun  versus  Wings
Mystics  versus  Dream
Sky  versus  Fever
Lynx  versus  Liberty
Sparks  versus  Mercury
Sun  versus  Wings
Fever  versus  Sparks
Storm  versus  Liberty
Sky  versus  Aces
Sun  

In [105]:
# Save weighted scoring networks

for t in team_names_dict:
    nx.write_weighted_edgelist(networks[team_names_dict[t]],path=f'WNBA/Scoring Networks/{t}_scoringNetwork.edgelist')

In [108]:
# Load weighted scoring networks

networks = {}

for team_id in team_ids_dict:
    g = nx.read_weighted_edgelist(path=f'WNBA/Scoring Networks/{team_ids_dict[team_id]}_scoringNetwork.edgelist')
    
    # Remove edges with 0 weight
    zero_edges = [e for e in g.edges(data=True) if e[2]['weight'] == 0]
    g.remove_edges_from(zero_edges)

    #Remove edges for players who didn't play
    g.remove_nodes_from(list(nx.isolates(g)))

    networks[team_id] = g

In [146]:
# Creating dataframe with edge weights for each pair of players
df = line
df['PLAYER_1'] = list(map(lambda x: x.split('-')[1],df['GROUP_ID'].to_list()))
df['PLAYER_2'] = list(map(lambda x: x.split('-')[2],df['GROUP_ID'].to_list()))

df.insert(0,'PLAYER_1',df.pop('PLAYER_1'))
df.insert(1,'PLAYER_2',df.pop('PLAYER_2'))

df = df.loc[df['MIN'] > 0,:]

# Create a dictionary to map the edge weight values for each pair of players to the respective row in the 
# 2-player lineup dataframe
pair_dict = {}
for team_id in team_ids_dict:
    g = networks[team_id]
    for e in g.edges(data=True):
        str1 = '-'+str(e[0])+'-'+str(e[1])+'-'
        str2 = '-'+str(e[1])+'-'+str(e[0])+'-'
        pair_dict[str1] = e[2]['weight']
        pair_dict[str2] = e[2]['weight']

df.insert(len(df.columns),'SCORING_EDGE_WEIGHT',pd.Series(df['GROUP_ID'].map(pair_dict)))

df = df.dropna(subset=['SCORING_EDGE_WEIGHT'])

df.to_csv('WNBA/2-Player Lineups.csv',index=False)

1091


In [12]:
# 

df = pd.read_csv('WNBA/2-Player Lineups.csv')

df.loc[df['MIN']>400,:].sort_values('SCORING_EDGE_WEIGHT',ascending=False)

,PLAYER_1,PLAYER_2,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,MIN,OFF_RATING,DEF_RATING,NET_RATING,...,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_PAINT,PCT_AST_FGM,PCT_UAST_FGM,OPP_FG3_PCT,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,SCORING_EDGE_WEIGHT
4,204319,1627668,-204319-1627668-,J. Loyd - B. Stewart,1611661328,SEA,908.2,107.8,95.1,12.7,...,0.163,0.151,0.412,0.766,0.234,0.318,0.487,0.199,0.199,1.232107
0,1628276,1628932,-1628276-1628932-,K. Plum - A. Wilson,1611661319,LVA,1029.0,112.1,98.3,13.8,...,0.137,0.179,0.397,0.649,0.351,0.344,0.494,0.215,0.165,1.225462
85,1627668,1629496,-1627668-1629496-,B. Stewart - E. Magbegor,1611661328,SEA,544.4,105.8,95.7,10.1,...,0.144,0.167,0.390,0.784,0.216,0.333,0.482,0.205,0.190,1.133358
10,203833,1628932,-203833-1628932-,C. Gray - A. Wilson,1611661319,LVA,853.6,113.8,98.1,15.7,...,0.138,0.176,0.401,0.651,0.349,0.342,0.497,0.197,0.168,1.112933
82,203399,1628878,-203399-1628878-,E. Delle Donne - A. Atkins,1611661322,WAS,560.5,106.0,96.0,10.0,...,0.067,0.173,0.403,0.683,0.317,0.343,0.495,0.250,0.163,1.099019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1628890,1629488,-1628890-1629488-,D. DeShields - B. Turner,1611661317,PHO,618.8,95.4,99.0,-3.7,...,0.141,0.174,0.425,0.682,0.318,0.357,0.493,0.274,0.175,0.612476
128,203828,1630101,-203828-1630101-,S. Dolson - C. Dangerfield,1611661313,NYL,426.0,97.8,97.6,0.2,...,0.112,0.151,0.426,0.725,0.275,0.329,0.473,0.210,0.148,0.607981
86,1628886,1629478,-1628886-1629478-,J. Canada - K. Samuelson,1611661320,LAS,543.0,97.5,107.7,-10.2,...,0.146,0.177,0.459,0.670,0.330,0.408,0.559,0.303,0.218,0.602210
48,1629482,1629488,-1629482-1629488-,S. Cunningham - B. Turner,1611661317,PHO,656.1,100.7,100.3,0.4,...,0.122,0.174,0.422,0.682,0.318,0.359,0.496,0.261,0.205,0.544124


In [13]:

text = ""

for team in list(team_names_dict.keys()):
    g = nx.read_weighted_edgelist(path=f'WNBA/Scoring Networks/{team}_scoringNetwork.edgelist')

    weights_dict = nx.get_edge_attributes(g,'weight')

    weights = list(weights_dict.values())
    max_weight = max(weights)
    second_largest_weight = sorted(set(weights))[-2]
    third_largest_weight = sorted(set(weights))[-3]


    key_list = list(weights_dict)
    position1 = weights.index(max_weight)
    position2 = weights.index(second_largest_weight)

    weights[position1] = third_largest_weight
    weights[position2] = third_largest_weight

    nx.set_edge_attributes(g, {key_list[position1]:{'weight':third_largest_weight},key_list[position2]:{'weight':third_largest_weight}})


    zero_edges = [(u,v) for (u,v,d) in g.edges(data=True) if d['weight'] == 0]
    zero_nodes = [n for n in g.nodes if g.degree(n, weight='weight') == 0]
    g.remove_edges_from(zero_edges)
    g.remove_nodes_from(zero_nodes)

    min_weight=min(list(nx.get_edge_attributes(g,'weight').values()))

    nx.set_edge_attributes(g,{v:(10.5*((k-min_weight+0.1)/(third_largest_weight-min_weight+0.1))) for v,k in nx.get_edge_attributes(g,'weight').items()},name='width')
    node_weight = {n:g.degree(n, weight='weight')/g.degree(n)*22.0 for n in g.nodes}
    nx.set_node_attributes(g,node_weight,name='size')

    net = Network(600,600)

    layout = nx.circular_layout(g)

    r = commonteamroster.CommonTeamRoster(season="2022", team_id=team_names_dict[team],league_id_nullable='10')
    time.sleep(1)
    roster1 = r.get_data_frames()[0]
    roster1 = roster1.set_index('PLAYER_ID',drop=False)
    roster = roster1[['PLAYER_ID','PLAYER']]

    IDs = roster['PLAYER_ID'].tolist()
    names = roster['PLAYER'].tolist()

    name_labels = {IDs[i] : names[i].split(" ")[-1] for i in range(len(IDs))}

    net.from_nx(g)
    for node in net.nodes:
        node["x"] = layout[node["id"]][0] * 260 +310
        node["y"] = layout[node["id"]][1] * 260+300
        deg = g.degree(node["id"],weight='weight') / g.degree(node["id"])
        position = roster1.loc[int(node["id"]),'POSITION']
        name = roster.loc[int(node["id"]),'PLAYER']
        node["title"] = name+"\n" +f'Connection-Adjusted Degree: {deg:.2f}'+\
            f"\nPosition:{position}"
        node["label"] = name_labels[int(node['id'])]
        node["font"] = '16px arial black'
        node["shape"] = 'image'
        node["image"] = f'https://ak-static.cms.nba.com/wp-content/uploads/headshots/wnba/latest/1040x760/{node["id"]}.png?im=FeatureCrop,size=(760,760)'
        node["brokenImage"] = 'https://www.wnba.com/wp-content/themes/wnba-parent/img/placeholder_default.png'
    net.toggle_physics(False)

    nonzero_e = [0] + [e for e in net.edges if e['weight'] > 0]

    colormap = plt.cm.Blues(np.linspace(0, 1, len(nonzero_e)))
    weights = [0] + [w for w in sorted(list(nx.get_edge_attributes(g,'weight').values())) if w]
    

    for edge in net.edges:
        c = colormap[weights.index(edge["weight"])].tolist()
        edge["color"] = f'rgb({c[0]*255},{c[1]*255},{c[2]*255})'
        to_name = roster.loc[int(edge['to']),'PLAYER']
        from_name = roster.loc[int(edge['from']),'PLAYER']
        edge["title"] = f"{from_name}-{to_name}\nPoints-per-minute: {edge['weight']:.2f}"
    

    text_to_add = net.generate_html()
    spot = text_to_add.index('left;')
    text_to_add = text_to_add[:spot+5] + " top: 5%; left: 25%; position: absolute;" + text_to_add[spot+5:]
    text_to_add = text_to_add.replace('mynetwork',f'{team}network')
    text_to_add = text_to_add.replace('<html>',f'<div id="{team}" class="inv"><h1>{team}</h1>')
    text_to_add = text_to_add.replace('</html>','</div>')

    text += text_to_add
    
    net.show("nx.html")

print(text)

<div id="Aces" class="inv"><h1>Aces</h1>
<head>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/vis-network@latest/styles/vis-network.css" type="text/css" />
<script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vis-network@latest/dist/vis-network.min.js"> </script>
<center>
<h1></h1>
</center>

<!-- <link rel="stylesheet" href="../node_modules/vis/dist/vis.min.css" type="text/css" />
<script type="text/javascript" src="../node_modules/vis/dist/vis.js"> </script>-->

<style type="text/css">

        #Acesnetwork {
            width: 600;
            height: 600;
            background-color: #ffffff;
            border: 1px solid lightgray;
            position: relative;
            float: left; top: 5%; left: 25%; position: absolute;
        }

        

        

        
</style>

</head>

<body>
<div id = "Acesnetwork"></div>


<script type="text/javascript">

    // initialize global variables.
    var edges;
    var nodes;
    var network; 
    var container